In [ ]:
import pandas as pd
import tqdm
from rdkit.Chem import MolFromSmiles, Lipinski # NumHDonors, NumHAcceptors

In [ ]:
df = pd.read_csv("zinc15_train_full.csv")
df = df.loc[:,('smiles', 'mwt', 'logp', 'length', 'cnt_ring', 'ba_bcl2', 'ba_bclxl', 'ba_bclw')]

In [ ]:
idx = (df.loc[:,'ba_bcl2'] > 9.069) & (df.loc[:,'ba_bclxl'] > 9.283) & (df.loc[:,'ba_bclw'] > 6.999)
df = df[idx].reset_index(drop=True)

In [ ]:
df

In [ ]:
for i, smi in tqdm.tqdm(enumerate(df.loc[:,'smiles'].values), total=len(df)):
    
    mol = MolFromSmiles(smi)
    
    df.loc[i,'donor'] = Lipinski.NumHDonors(mol)
    df.loc[i,'acceptor'] = Lipinski.NumHAcceptors(mol)

In [ ]:
for i in tqdm.trange(len(df)):
    
    df.loc[i,'violation_Ro5'] = 0
    
    df.loc[i,'violation_Ro5'] += 0 if df.loc[i,'donor'] < 5 else 1
    df.loc[i,'violation_Ro5'] += 0 if df.loc[i,'acceptor'] < 10 else 1
    df.loc[i,'violation_Ro5'] += 0 if df.loc[i,'mwt'] < 500 else 1
    df.loc[i,'violation_Ro5'] += 1 if df.loc[i,'logp'] > 5 else 0

In [ ]:
df.loc[:,'violation_Ro5'].value_counts() / len(df)